# Enrichr

> Fix, clean markdown headings and enrich it with figures description, ...

This module aims to fix and enrich markdown headings from OCR'd PDF files by:

1. Fixing heading hierarchy that was corrupted during OCR
2. Adding page numbers to headings for better navigation
3. Enriching figure references with descriptive text and creating a table of figures

In [ ]:
#| default_exp enrichr

In [ ]:
#| export
from pathlib import Path
import os
import re
import json
import shutil
import time
from functools import partial
from tqdm import tqdm
from dotenv import load_dotenv
from fastcore.all import *
import dspy
from pydantic import BaseModel
from typing import List
from litellm import completion
import base64
from rich import print

In [ ]:
#| export
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

In [ ]:
#| exports
cfg = AttrDict({
    'enhanced_dir': 'enhanced',
    'enriched_dir': 'enriched',
    'lm': 'gemini/gemini-2.0-flash-exp',
    'api_key': GEMINI_API_KEY,
    'max_tokens': 8192,
    'track_usage': False,
    'img_dir': 'img'
})

In [ ]:
#| export
src_dir = Path("../_data/md_library/49d2fba781b6a7c0d94577479636ee6f")

## Fixing Markdown Headings

In [ ]:
#| eval: false
doc = src_dir / 'abridged_evaluation_report_final_olta_ndoja_pdf'
pages = doc.ls(file_exts=".md").sorted(key=lambda p: int(p.stem.split('_')[1])); pages

(#31) [Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/page_1.md'),Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/page_2.md'),Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/page_3.md'),Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/page_4.md'),Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/page_5.md'),Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/page_6.md'),Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/page_7.md'),Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/page_8.md'),Path('../_data/md_library/49d2fba781b6a7c

In [ ]:
#| exports
def setup_enhanced_dir(
    src_dir, # Source directory path
    enhanced_dir_name=cfg.enhanced_dir # Name of enhanced subdirectory
    ):
    "Create enhanced directory and copy all markdown files to it"
    src_path = Path(src_dir)
    enhanced_path = src_path / enhanced_dir_name
    enhanced_path.mkdir(exist_ok=True)
    for f in src_path.ls(file_exts=".md"): shutil.copy(f, enhanced_path)
    return enhanced_path

For instance:

In [ ]:
#| eval: false
setup_enhanced_dir(doc, 'enhanced')

Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/enhanced')

In [ ]:
#| exports
def get_hdgs(md_txt): return re.findall(r'^#+.*$', md_txt, re.MULTILINE)

In [ ]:
#| exports
def get_hdgs_with_pages(
    pages: list[Path] # List of pages
    ):
    "Get headings and the page number they are on"
    headings = []
    for i, page in enumerate(pages, 1):  # page numbers start at 1
        page_headings = get_hdgs(page.read_text())
        # add each heading with its page number
        for o in page_headings:
            headings.append({'heading': o, 'page': i})
    return headings

In [ ]:
#| eval: false
hdgs = get_hdgs_with_pages(pages); hdgs[:5]

[{'heading': '# **PPMi**', 'page': 1},
 {'heading': '# CONTENTS ', 'page': 3},
 {'heading': '# 1. Introduction ', 'page': 4},
 {'heading': '# 2. Background of the JI-HoA ', 'page': 5},
 {'heading': '### 2.1. Context and design of the JI-HoA', 'page': 5}]

In [ ]:
#| eval: false
toc = L([get_hdgs(p.read_text()) for p in pages]).concat(); toc[:5]

(#5) ['# **PPMi**','# CONTENTS ','# 1. Introduction ','# 2. Background of the JI-HoA ','### 2.1. Context and design of the JI-HoA']

In [ ]:
#| exports
def format_hdgs(
    hdgs: list[dict] # List of headings with page numbers
    ):
    "Format headings with page numbers"
    formatted = []
    page_positions = {}
    
    for item in hdgs:
        page = item['page']
        page_positions[page] = page_positions.get(page, 0) + 1
        formatted.append(f"{item['heading']} (Page {page}, Position {page_positions[page]})")
    
    return "\n".join(formatted)

In [ ]:
#| eval: false
print(format_hdgs(hdgs)[:500])

# **PPMi** (Page 1, Position 1)
# CONTENTS  (Page 3, Position 1)
# 1. Introduction  (Page 4, Position 1)
# 2. Background of the JI-HoA  (Page 5, Position 1)
### 2.1. Context and design of the JI-HoA (Page 5, Position 2)
# 2.2. External factors affecting the implementation of the JI  (Page 7, Position 1)
# 3. Methodology  (Page 8, Position 1)
# 4. Findings  (Page 10, Position 1)
### 4.1. Relevance (Page 10, Position 2)
### 4.1.1. Relevance of programme activities for migrants, returnees, and comm

In [ ]:
#| exports
lm = dspy.LM(cfg.lm, api_key=cfg.api_key)
dspy.configure(lm=lm)
dspy.settings.configure(track_usage=cfg.track_usage)

In [ ]:
#| exports
class HeadingResult(BaseModel):
    old: str
    page: int
    position: int
    new: str
    changed: bool  # True if correction was made

In [ ]:
#| exports
class FixHeadingHierarchy(dspy.Signature):
    """Fix markdown heading hierarchy by analyzing the document's numbering patterns:
    - Detect numbering scheme (1.2.3, I.A.1, A.1.a, etc.)
    - Apply hierarchy levels based on nesting depth: # for top level, ## for second level, ### for third level
    - When a section number is lower than a previously seen number at the same level (e.g., seeing '2.' after '3.1'), it's likely a subsection or list item, not a main section
    - Unnumbered headings: keep as-is if at document boundaries, treat as subsections if within numbered sections
    - Return ALL headings with their corrected form
    """
    
    headings_with_pages: str = dspy.InputField(desc="List of headings with page numbers")
    results: List[HeadingResult] = dspy.OutputField(desc="All headings with corrections and change status")

In [ ]:
#| exports
def fix_md(
    hdgs: list[dict], # List of headings with page numbers
    track_usage: bool=cfg.track_usage,
    ):
    "Fix markdown headings"
    lm = dspy.LM(cfg.lm, api_key=cfg.api_key, max_tokens=cfg.max_tokens)
    dspy.configure(lm=lm)
    dspy.settings.configure(track_usage=track_usage)

    inp = format_hdgs(hdgs)
    fix_hdgs = dspy.ChainOfThought(FixHeadingHierarchy)
    result = fix_hdgs(headings_with_pages=inp)
    return result

In [ ]:
#| eval: false
result = fix_md(hdgs, track_usage=True)
print("Result:", result)
print("Usage:", result.get_lm_usage())

Result: Prediction(
    reasoning='The document uses a mixed numbering scheme.  The main sections are numbered 1, 2, 3, 4, 5.  
Subsections are numbered with decimal points (e.g., 4.1, 4.1.1).  However, there are inconsistencies. Some 
subsections are incorrectly formatted as top-level headings (e.g., "2.2").  Some unnumbered headings are present 
within numbered sections, which should be treated as subsections.  The algorithm will correct the heading levels 
based on the numbering scheme and context, ensuring a consistent hierarchy.  Unnumbered headings within numbered 
sections will be treated as subsections of the nearest numbered heading. Unnumbered headings at the beginning or 
end of the document will remain as top-level headings.  Headings that appear to be text within a section (like the 
overall performance scores) will be left as is.',
    results=[HeadingResult(old='# **PPMi**', page=1, position=1, new='# PPMi', changed=True), HeadingResult(old='# 
CONTENTS', page=3, position=1, new='# CONTENTS', changed=True), HeadingResult(old='# 1. Introduction', page=4, 
position=1, new='# 1. Introduction', changed=False), HeadingResult(old='# 2. Background of the JI-HoA', page=5, 
position=1, new='# 2. Background of the JI-HoA', changed=False), HeadingResult(old='### 2.1. Context and design of 
the JI-HoA', page=5, position=2, new='### 2.1. Context and design of the JI-HoA', changed=False), 
HeadingResult(old='# 2.2. External factors affecting the implementation of the JI', page=7, position=1, new='### 
2.2. External factors affecting the implementation of the JI', changed=True), HeadingResult(old='# 3. Methodology',
page=8, position=1, new='# 3. Methodology', changed=False), HeadingResult(old='# 4. Findings', page=10, position=1,
new='# 4. Findings', changed=False), HeadingResult(old='### 4.1. Relevance', page=10, position=2, new='### 4.1. 
Relevance', changed=False), HeadingResult(old='### 4.1.1. Relevance of programme activities for migrants, 
returnees, and communities', page=10, position=3, new='#### 4.1.1. Relevance of programme activities for migrants, 
returnees, and communities', changed=False), HeadingResult(old='### 4.1.1.1 Needs of migrants', page=10, 
position=5, new='##### 4.1.1.1 Needs of migrants', changed=True), HeadingResult(old='### 4.1.1.2 Needs of 
returnees', page=10, position=6, new='##### 4.1.1.2 Needs of returnees', changed=True), HeadingResult(old='# 
4.1.1.3 Needs of community members', page=12, position=1, new='##### 4.1.1.3 Needs of community members', 
changed=True), HeadingResult(old="### 4.1.2. Programme's relevance to the needs of stakeholders", page=12, 
position=2, new="### 4.1.2. Programme's relevance to the needs of stakeholders", changed=False), 
HeadingResult(old='### 4.1.2.1 Needs of governments', page=12, position=3, new='#### 4.1.2.1 Needs of governments',
changed=True), HeadingResult(old='# 4.1.2.2 Needs of other stakeholders', page=13, position=1, new='#### 4.1.2.2 
Needs of other stakeholders', changed=True), HeadingResult(old='### 4.2. Coherence', page=13, position=2, new='### 
4.2. Coherence', changed=False), HeadingResult(old="# 4.2.1. The JI-HoA's alignment with the objectives and 
standards of IOM, and objectives of the EU", page=14, position=1, new="#### 4.2.1. The JI-HoA's alignment with the 
objectives and standards of IOM, and objectives of the EU", changed=False), HeadingResult(old='### 4.2.2. Alignment
with other initiatives', page=14, position=2, new='#### 4.2.2. Alignment with other initiatives', changed=False), 
HeadingResult(old='# 4.3. Effectiveness', page=16, position=1, new='# 4.3. Effectiveness', changed=False), 
HeadingResult(old='### 4.3.1. Specific Objective 1: Partner countries and relevant stakeholders developed or 
strengthened evidence-based return and reintegration procedures', page=16, position=2, new='### 4.3.1. Specific 
Objective 1: Partner countries and relevant stakeholders developed or strengthened evidence-based return and 
reintegration procedures'

Usage:
{}

In [ ]:
#| exports
def group_corrections_by_page(
    results: list[HeadingResult], # List of headings with corrections and change status
    ):
    "Group HeadingResult corrections by page number into dict with page nums as keys"
    page_groups = {}
    for result in results:
        page = result.page
        if page not in page_groups:
            page_groups[page] = []
        page_groups[page].append(result)
    return page_groups

In [ ]:
#| eval: false
group_corrections_by_page(result.results)

{1: [HeadingResult(old='# **PPMi**', page=1, position=1, new='# PPMi', changed=True)],
 3: [HeadingResult(old='# CONTENTS', page=3, position=1, new='# CONTENTS', changed=True)],
 4: [HeadingResult(old='# 1. Introduction', page=4, position=1, new='# 1. Introduction', changed=False)],
 5: [HeadingResult(old='# 2. Background of the JI-HoA', page=5, position=1, new='# 2. Background of the JI-HoA', changed=False),
  HeadingResult(old='### 2.1. Context and design of the JI-HoA', page=5, position=2, new='### 2.1. Context and design of the JI-HoA', changed=False)],
 7: [HeadingResult(old='# 2.2. External factors affecting the implementation of the JI', page=7, position=1, new='### 2.2. External factors affecting the implementation of the JI', changed=True)],
 8: [HeadingResult(old='# 3. Methodology', page=8, position=1, new='# 3. Methodology', changed=False)],
 10: [HeadingResult(old='# 4. Findings', page=10, position=1, new='# 4. Findings', changed=False),
  HeadingResult(old='### 4.1. Releva

In [ ]:
#| exports
def apply_corrections_to_page(
    page_nb, # Page number
    corrections, # List of corrections
    enhanced_path, # Path to enhanced directory
    ):
    "Apply corrections to a page in the enhanced directory"
    page_file = enhanced_path / f"page_{page_nb}.md"
    lines = page_file.read_text().splitlines()
    corrections_copy = corrections.copy()
    
    for i, line in enumerate(lines):
        for correction in corrections_copy:
            if line.strip() == correction.old.strip():
                lines[i] = f"{correction.new} .... page {page_nb}"
                corrections_copy.remove(correction)
                break
            
    page_file.write_text('\n'.join(lines))

In [ ]:
#| eval: false
enhanced_path = doc / cfg.enhanced_dir
apply_corrections_to_page(5, result.results, enhanced_path)

In [ ]:
#| exports
def apply_all_corrections(
    results, # List of headings with corrections and change status
    enhanced_path, # Path to enhanced directory
    ):
    "Apply all corrections to the pages in enhanced directory"
    grouped = group_corrections_by_page(results)
    for page_nb, corrections in grouped.items(): 
        apply_corrections_to_page(page_nb, corrections, enhanced_path)

In [ ]:
#| eval: false
apply_all_corrections(result.results, enhanced_path)

In [ ]:
#| exports
def fix_doc_hdgs(
    src_dir, # Path to the folder containing the document
    force=False, # Whether to overwrite the existing enhanced directory
    ):
    "Process the document directory"
    src_path = Path(src_dir)
    enhanced_path = src_path / cfg.enhanced_dir
    
    if enhanced_path.exists() and not force:
        print(f"Enhanced directory '{cfg.enhanced_dir}' already exists. Use force=True to overwrite.")
        return
    if enhanced_path.exists() and force: 
        shutil.rmtree(enhanced_path)
    
    enhanced_path = setup_enhanced_dir(src_dir)
    pages = enhanced_path.ls(file_exts=".md").sorted(key=lambda p: int(p.stem.split('_')[1]))
    result = fix_md(get_hdgs_with_pages(pages))
    apply_all_corrections(result.results, enhanced_path)

In [ ]:
#| eval: false
print(doc)
fix_doc_hdgs(doc, force=True)

../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf

## Enrich with figures description

In [ ]:
#| eval: false
doc = src_dir / 'abridged_evaluation_report_final_olta_ndoja_pdf/enhanced'
pages = doc.ls(file_exts=".md").sorted(key=lambda p: int(p.stem.split('_')[1])); pages

(#31) [Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/enhanced/page_1.md'),Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/enhanced/page_2.md'),Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/enhanced/page_3.md'),Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/enhanced/page_4.md'),Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/enhanced/page_5.md'),Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/enhanced/page_6.md'),Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/enhanced/page_7.md'),Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_

In [ ]:
#| exports
def has_images(page_path):
    content = Path(page_path).read_text()
    return bool(re.search(r'!\[[^\]]*\]\([^)]+\)', content))

For instance:

In [ ]:
#| eval: false
[page for page in pages if has_images(page)]

[Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/enhanced/page_1.md'),
 Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/enhanced/page_6.md'),
 Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/enhanced/page_8.md'),
 Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/enhanced/page_12.md'),
 Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/enhanced/page_18.md'),
 Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/enhanced/page_21.md'),
 Path('../_data/md_library/49d2fba781b6a7c0d94577479636ee6f/abridged_evaluation_report_final_olta_ndoja_pdf/enhanced/page_24.md')]

In [ ]:
#| exports
class MarkdownPage: 
    "A class to represent a markdown page"
    def __init__(self, path): self.path = Path(path)

In [ ]:
#| exports
class ImgRef(AttrDict):
    "A class to represent a image reference"
    def __repr__(self):
        clean_context = self.context.replace('\n', ' ')[:50] + "..."
        fields = [f"filename='{self.filename}'", f"context='{clean_context}'"]
        if hasattr(self, 'is_relevant'): fields.append(f"is_relevant={self.is_relevant}")
        if hasattr(self, 'reason'): fields.append(f"reason={self.reason}")
        # ... add other fields if present
        return f"ImgRef({', '.join(fields)})"


In [ ]:
#| exports
@patch
def find_img_refs(
    self:MarkdownPage, # Markdown page of interest
    context_lines: int = 3, # Number of lines of context to include around the image
    ):
    "Find all image references in the markdown page and include the context around the image"
    content = self.path.read_text()
    lines = content.splitlines()
    results = []
    
    for i, line in enumerate(lines):
        if re.search(r'!\[[^\]]*\]\(([^)]+)\)', line):
            # Extract context around this line
            start = max(0, i - context_lines)
            end = min(len(lines), i + context_lines + 1)
            context = '\n'.join(lines[start:end])
            
            # Extract image filename
            match = re.search(r'!\[[^\]]*\]\(([^)]+)\)', line)
            results.append(ImgRef({
                "filename": match.group(1),
                "context": context
            }))
    
    return results

For instance: 

In [ ]:
#| eval: false
for page in pages: 
    img_refs = MarkdownPage(page).find_img_refs()
    if img_refs: print(f"In {page.stem}: {img_refs}")

In page_1: [ImgRef(filename='img-0.jpeg', context=' Final Evaluation Report, 17 March 2023  !

In page_6: [ImgRef(filename='img-1.jpeg', context='FIGURE 1. IOM'S INTEGRATED APPROACH TO REINTEGRATI...')]

In page_8: [ImgRef(filename='img-2.jpeg', context='The evaluation followed the Organisation of Econom...')]

In page_12: [ImgRef(filename='img-3.jpeg', context='According to the community participation survey ad...')]

In page_18: [ImgRef(filename='img-4.jpeg', context='The number of stakeholders (state and non-state) i...')]

In page_21: [ImgRef(filename='img-5.jpeg', context='origin. This target was exceeded in Somalia ( $80 ...')]

In page_24: [ImgRef(filename='img-6.jpeg', context='The financial resources were sufficient to meet th...')]

In [ ]:
#| exports
class ImageRelevance(dspy.Signature):
    """Determine if an image contains substantive content for document understanding.
    
    RELEVANT: Charts, graphs, diagrams, figures, tables, screenshots, flowcharts
    IRRELEVANT: Logos, cover images, decorative elements, headers, footers
    """
    img_filename: str = dspy.InputField()
    surrounding_context: str = dspy.InputField(desc="Text context around the image")
    is_relevant: bool = dspy.OutputField(desc="True only for substantive content like data visualizations")
    reason: str = dspy.OutputField(desc="Brief explanation of decision")


In [ ]:
#| exports
@patch
def classify_imgs(
    self:MarkdownPage, # Markdown page of interest
    img_refs: list[ImgRef], # List of image references
    ):
    "Classify images in the markdown page"
    classifier = dspy.ChainOfThought(ImageRelevance)
    for img_ref in img_refs:
        result = classifier(
            img_filename=img_ref.filename,
            surrounding_context=img_ref.context,
            page_nb=1  # We could make this dynamic if needed
        )
        img_ref.is_relevant = result.is_relevant
        img_ref.reason = result.reason
    return img_refs

For instance:

In [ ]:
#| eval: false
img_refs = MarkdownPage(pages[0]).find_img_refs(); print(img_refs)
md_page = MarkdownPage(pages[5]) 
img_refs = md_page.find_img_refs()
clf_img_refs = md_page.classify_imgs(img_refs)
print(clf_img_refs)

[ImgRef(filename='img-0.jpeg', context=' Final Evaluation Report, 17 March 2023  ![img-0.j...')]

[
    ImgRef(filename='img-1.jpeg', context='FIGURE 1. IOM'S INTEGRATED APPROACH TO REINTEGRATI...', 
is_relevant=True, reason=The image is a figure illustrating an integrated approach, likely a diagram or flowchart.)
]

In [ ]:
#| exports
def describe_img(
    img_path: Path, # Path to the image
    context: str, # Context of the image
    api_key: str = GEMINI_API_KEY, # API key for the Gemini model
    model: str = cfg.lm, # Model to use
    ):
    "Describe an image using an LLM"
    with open(img_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')
    
    # Auto-detect image format
    img_format = img_path.suffix.lower().replace('.', '')
    if img_format == 'jpg': img_format = 'jpeg'
    
    prompt = f"""Provide a concise paragraph description of this image for evaluation report analysis. Include: type of content, main topic, key data/statistics, trends, and takeaways. Write as flowing text, not numbered points. Context: {context}"""
    response = completion(
        model=model,
        messages=[{
            "role": "user", 
            "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/{img_format};base64,{base64_image}"}}
            ]
        }],
        api_key=api_key
    )
    return response.choices[0].message.content

In [ ]:
#| exports
@patch
def describe_imgs(
    self:MarkdownPage, # Markdown page of interest
    img_refs: list[ImgRef], # List of image references
    img_dir: str # Image directory
    ):
    "Describe images in the markdown page"
    for img_ref in img_refs:
        if img_ref.is_relevant:
            img_path = Path(img_dir) / img_ref.filename
            description = describe_img(img_path, img_ref.context, GEMINI_API_KEY)
            img_ref.description = description
    return img_refs

For instance:

In [ ]:
#| eval: false
md_page = MarkdownPage(pages[5]) 
img_refs = md_page.find_img_refs()
clf_img_refs = md_page.classify_imgs(img_refs)
img_refs_desc = md_page.describe_imgs(clf_img_refs, doc.parent / 'img')
print(img_refs_desc[0].description)

Figure 1, sourced from the IOM Reintegration Handbook, presents a process diagram outlining the IOM's integrated 
approach to reintegration. The figure visually separates the process into three intersecting societal levels: 
Individual, Community, and Structural. The outer ring of the diagram emphasizes Economic, Social, and Psychosocial 
factors that are monitored, evaluated, and inform learning within the reintegration process. The individual level 
focuses on tailored assistance, especially for vulnerable migrants, with preparation, flexibility, and follow-up 
being key. The community level centers on faster inclusion and strengthened resilience through social networks, 
joint initiatives, and partnerships. Lastly, the structural level stresses alignment with national and local 
development policies through capacity building, local ownership, and essential services. The core takeaway is the 
cyclical and multi-faceted nature of reintegration, highlighting the need for tailored, community-oriented, and 
structurally supported approaches.

In [ ]:
#| exports
@patch
def replace_imgs_with_desc(
    self:MarkdownPage, # Markdown page of interest
    img_refs, # List of image references
    enriched_dir: str = cfg.enriched_dir, # Enriched directory
    ):
    "Replace images with their descriptions in the markdown page"
    enriched_path = self.path.parent.parent / enriched_dir
    enriched_path.mkdir(exist_ok=True)
    
    content = self.path.read_text()
    for img_ref in img_refs:
        if img_ref.is_relevant and hasattr(img_ref, 'description'):
            pattern = f'!\\[[^\\]]*\\]\\({re.escape(img_ref.filename)}\\)'
            content = re.sub(pattern, img_ref.description, content)
    
    enriched_file = enriched_path / self.path.name
    enriched_file.write_text(content)
    return enriched_file

In [ ]:
#| exports
def copy_page_to_enriched(
    page, # Page to copy
    enriched_dir: str = cfg.enriched_dir, # Enriched directory
    ):
    "Copy a page to the enriched directory"
    enriched_path = page.parent.parent / enriched_dir
    enriched_path.mkdir(exist_ok=True)
    return shutil.copy(page, enriched_path)

In [ ]:
#| exports
def process_single_page(
    page, # Page to process
    img_dir, # Image directory
    enriched_dir: str = cfg.enriched_dir, # Enriched directory
    ):
    "Process a single page"
    md_page = MarkdownPage(page)
    # Pipeline: find → classify → describe → replace
    img_refs = md_page.find_img_refs()
    
    if not img_refs: return copy_page_to_enriched(page, enriched_dir)
    
    classified_refs = md_page.classify_imgs(img_refs)
    described_refs = md_page.describe_imgs(classified_refs, img_dir)
    return md_page.replace_imgs_with_desc(described_refs)

In [ ]:
#| exports
def enrich_images(
    pages_dir, # Pages directory
    img_dir, # Image directory
    n_workers=2, # Number of workers
    ):
    "Enrich images in the pages directory"
    pages = Path(pages_dir).ls(file_exts=".md")
    
    pages_with_imgs = []
    for page in pages:
        if has_images(page):
            pages_with_imgs.append(page)
        else:
            copy_page_to_enriched(page)
    
    if pages_with_imgs:
        process_fn = partial(process_single_page, img_dir=img_dir)
        parallel(process_fn, pages_with_imgs, n_workers=n_workers, threadpool=True, progress=True)
        
    print(f"✓ Processed {len(pages)} pages ({len(pages_with_imgs)} with images)")

In [ ]:
#| eval: false
enrich_images(doc, doc.parent / 'img', n_workers=2)

✓ Processed 31 pages (7 with images)